### Density Estimation based on TAO
https://github.com/iworld1991/ExpProject/blob/master/workingfolder/python/DensityEst.ipynb
Following Engelberg et al 2009: https://www.tandfonline.com/doi/pdf/10.1198/jbes.2009.0003?needAccess=true

Three cases
1. 3+ intervales with positive probabilities, to be fitted with a generalized beta distribution
2. exactly 2 adjacent intervals with positive probabilities, to be fitted with a triangle distribution
3. one interval only, to be fitted with a uniform distribution

Problems:
2. Some bins in between are 0
4. There are more cases which I cannot really explain. Maybe too much variation across bins and the beta distribution cannot handle that?

In [1]:
from scipy.stats import gamma
from scipy.stats import beta 
from scipy.stats import triang # new
from scipy.stats import uniform # new
import matplotlib.pyplot as plt
from scipy.optimize import minimize
import numpy as np
import pandas as pd

import math as math

In [2]:
# We need that for the 90-10 ratio:
def quantile(x,quantiles):
    xsorted = sorted(x)
    qvalues = [xsorted[int(q * len(xsorted))] for q in quantiles]
    return qvalues

### Case 1. Generalized Beta Distribution¶

In [3]:
def GeneralizedBetaEst(bin,probs):
    """
    This fits a histogram with positive probabilities in at least 3 bins to a generalized beta distribution.
    Depending on if there is open-ended bin on either side with positive probability, 
       the estimator decides to estimate 2 or 4 parameters, respectively. 
       
    paramters
    ---------
    bin:  ndarray, (n+1) x 1 
          positions for n bins in the histograms 
          
    probs:  ndarrray n x 1
          each entry is a probability for each of the n bins given by the surveyee, between 0 and 1
                 
    returns
    -------
    moments:  ndarray of 2 or 4  
              2:  alpha and beta 
              4:  alpha, beta, lb, ub, e.g. lb=0 and ub=1 for a standard beta distribution
    """
    # n+1 bins and n probs, both are arrays
    if sum([probs[i]>0 for i in range(len(bin)-1)])<3:
        print("Warning: at least three bins with positive probs are needed")
        para_est=[]
    if math.fsum(probs)!=1:
        print("probs need to sum up to 1")
        para_est=[]
    else:
        cdf = np.cumsum(probs)
        pprob=[i for i in range(len(bin)-1) if probs[i]>0]
        lb=bin[min(pprob)]
        print("lower bound is "+str(lb))
        ub=bin[max(pprob)+1]
        print("upper bound is "+str(ub))
        
        x0_2para = (2,1)
        x0_4para = (2,1,0,1) 
        def distance2para(paras2): # if there is no open-ended bin with positive probs 
            a,b=paras2
            distance= sum((beta.cdf(bin[1:],a,b,loc=lb,scale=ub-lb)-cdf)**2)
            return distance
        def distance4para(paras4): # if either on the left or right side one open-ended bin is with postive probs
            a,b,lb,ub=paras4
            distance= sum((beta.cdf(bin[1:],a,b,loc=lb,scale=ub-lb)-cdf)**2)
            return distance
        if lb==bin[0] and ub==bin[-1]:
            result = minimize(distance4para,x0_4para,
                                method='CG')
            if result['success']:
                para_est = result['x']
            else:
                para_est = []
            
        else:
            result = minimize(distance2para,x0_2para,
                                method='CG')
            para_est = result['x']
            if result['success']:
                para_est = result['x']
                scale = ub-lb
                para_est = np.concatenate([para_est,
                                          np.array([lb,scale])])
            else:
                para_est = []
        return para_est   # could be 2 or 4 parameters 

In [4]:
def GeneralizedBetaStats(a,b,lb,ub):
    """
    This function computes the moments of a generalized beta distribution, mean and variance for now. 
    
    parameters
    ----------
    a, b, lb, ub: floats #changed to a,b instead of alpha and beta
    
    returns
    -------
    dict:  2 keys
           mean, float 
           variance, float
           standard deviation, float
           ratio 90 10, float
    """
    # lb=0 and ub=1 for a standard beta distribution
    #mean = lb + (ub-lb)*a/(a+b)
    #var = (ub-lb)**2*a*b/((a+b)**2*(a+b+1))
    mean, var, skew, kurt = beta.stats(a, 
                                       b, 
                                       loc=lb, 
                                       scale=ub-lb, 
                                       moments='mvsk')
    std = np.sqrt(var)
    # ratio
    x = np.linspace(beta.ppf(0.01, a, b), beta.ppf(0.99, a, b), 100)
    quantiles = quantile(x,[0.1, 0.9])
    iqr1090 = quantiles[0]-quantiles[1]
    return {"mean": mean,
            "variance":var, 
            "std": std, 
            "iqr1090": iqr1090,
            "skewness":skew,
            "kurtosis":kurt}

### Case 2. Isosceles Triangle distribution

In [5]:
def TriangleEst(bin,probs):
    """
    The function fits histograms with exactly two adjacent 
       bins with positive probabilitie to a triangular distribution.
    It genetes the bounds of the isoceles triangle distribution. 
    
    paramters
    ---------
    bin:  ndarray, (n+1) x 1 
          positions for n bins in the histograms 
          
    probs:  ndarrray n x 1
          each entry is a probability for each of the n bins given by the surveyee, between 0 and 1
    
    returns
    --------
    dict: 3 keys
          lb: float, left bound 
          ub: float, right bound
          h:  float, height of the triangle
          c: float, mode
    
    """
    if sum([probs[i]>0 for i in range(len(bin)-1)])==2:
        print("There are two bins with positive probs")
        pprobadj = [i for i in range(1,len(bin)-3) if probs[i]>0 and probs[i+1]>0]   # from 1 to -3 bcz excluding the open-ended on the left/right
        if sum(pprobadj)>0:
            print('The two intervals are adjacent and not open-ended')
            min_i = min(pprobadj)
            #print(min_i)
            #print(probs[min_i])
            #print(probs[min_i+1])
            #print(pprobadj[0])
            #print(pprobadj[0]+2)
            #print(probs[min_i] > probs[min_i+1])
            #print(bin[pprobadj[0]])
            #print(bin[pprobadj[0]+2])
            if probs[min_i] > probs[min_i+1]:
                alf = probs[min_i+1]
                lb = bin[pprobadj[0]]
                scl = bin[pprobadj[0]+1]-bin[pprobadj[0]]
                t = scl*(alf/(1-alf) +np.sqrt(alf)/(1-alf))
                ub = bin[pprobadj[0]+1]+t 
                h = 2/(t+bin[pprobadj[0]+1]-bin[pprobadj[0]])
                c = ub - 2/h
            if probs[min_i] < probs[min_i+1]:
                alf = probs[min_i]
                ub = bin[pprobadj[0]+2]
                scl = bin[pprobadj[0]+2]-bin[pprobadj[0]+1]
                t = scl*(alf/(1-alf) + np.sqrt(alf)/(1-alf))
                lb = bin[pprobadj[0]+1]-t  
                h = 2/(t+bin[pprobadj[0]+2]-bin[pprobadj[0]+1])
                c = ub - 2/h
            if probs[min_i] == probs[min_i+1]:
                ub=bin[pprobadj[0]]
                lb=bin[pprobadj[0]+2]
                h = 2/(ub-lb)
                c = ub - 2/h
        else:
            lb = np.nan
            ub = np.nan
            h = np.nan
            c = np.nan
            print('Warning: the two intervals are not adjacent or are open-ended')
    return {'lb':lb,'ub':ub,"height":h,"mode":c}

In [6]:
def TriangleStats(lb,ub,c):
    """
    parameters
    ----------
    lb and ub:  float, left and right bounds of the triangle distribution
    c : float, mode
    
    returns
    -------
    dict:  2 keys for now
           mean: estimated mean
           variance: estimated variance
           std, float 
           irq1090, float  
    """
    
    mean = (lb+ub)/2
    var = (lb**2+ub**2+(lb+ub)**2/4-lb*(lb+ub)/2-ub*(lb+ub)/2-lb*ub)/18
    std = np.sqrt(var)
    skew = 0
    kurt = -3/5
#     x = np.linspace(triang.ppf(0.01, c, lb, ub), triang.ppf(0.99, c, lb, ub), 100)
#     x = np.linspace(triang.ppf(0.01, c),
#                 triang.ppf(0.99, c), 100)
    if lb > c:
        lb = c
    x = np.random.triangular(lb, c, ub, 100)
    p10 = np.percentile(x, 10)
    p90 = np.percentile(x, 90)
    iqr1090 = p90 - p10
#     quantiles = quantile(x,[0.1, 0.9])
#     iqr1090 = quantiles[0]-quantiles[1]
    return {"mean": mean,
            "variance":var, 
            "std": std, 
            "iqr1090": iqr1090,
           'skewness':skew,
            'kurtosis':kurt}

### Case 3. Uniform Distribution

In [7]:
def UniformEst(bins,probs):
    """
    This function fits a histogram with only one bin of positive probability to a uniform distribution.
    
    paramters
    ---------
    bins:  ndarray, (n+1) x 1 
          positions for n bins in the histograms 
          
    probs:  ndarrray n x 1
          each entry is a probability for each of the n bins given by the surveyee, between 0 and 1
    
    returns
    --------
    dict: 2 keys
          lb and ub, float. the left and right bounds of the uniform distribution
    """    
    pos_bins = []
    # find non zero positions
    pos_entry = np.argwhere(probs!=0)
            
    for i in pos_entry:
        pos_bins.append(probs[i])
    pos_bins = np.hstack(pos_bins)   # clean    

    if len(pos_entry)>1:
        for i in range(len(pos_entry)-1) :
            if pos_entry[i+1] - pos_entry[i] !=1:
                pos_bins = [] 
               
    if len(pos_bins)==1:
        print('length pos bins is 1')
        lb = bins[pos_entry[0]]
        lb = lb[0] # into float
        ub = bins[pos_entry[0] + 1]
        ub = ub[0] # into float
    if len(pos_bins)>1:
        print('length pos bins is > 1')
        lb = bins[pos_entry[0]]
        lb = lb[0] # into float
        ub = bins[pos_entry[0] + len(pos_bins)] 
        ub = ub[0] # into float
    if len(pos_bins)==0:
        lb=np.nan
        ub=np.nan

    return {"lb":lb,"ub":ub}

In [8]:
def UniformStats(lb,ub):
    """
    The function computes the moment of a uniform distribution.
    
    parameters
    ----------
    lb and ub, floats, left and right bounds of uniform distribution
    
    returns
    --------
    dict:  2 keys for now
           mean: estimated mean
           variance: estimated variance 
           std, float 
           irq1090, float  
    """
    
    if lb is not np.nan and ub is not np.nan: 
        mean = (lb+ub)/2
        var = (ub-lb)**2/12
        std = np.sqrt(var)
#         x = np.linspace(uniform.ppf(lb),uniform.ppf(ub), 100)
        x = np.linspace(lb, ub, 100)
        p10 = np.percentile(x, 10)
        p90 = np.percentile(x, 90)
        iqr1090 = p90 - p10
        skew = 0
        kurt = -5/6
    else:
        mean=np.nan
        var=np.nan
        std = np.nan
        iqr1090 = np.nan
        skew = np.nan
        kurt = np.nan
    return {"mean": mean,
            "variance":var, 
            "std": std, 
            "iqr1090": iqr1090,
            "skewness":skew,
            "kurtosis":kurt}

In [9]:
## This is the synthesized density estimation function
def SynDensityStat(bin,probs):
    """
    Synthesized density estimate module:
    It first detects the shape of histograms
    Then accordingly invoke the distribution-specific tool.
    
    paramters
    ---------
    bin:  ndarray, (n+1) x 1 
          positions for n bins in the histograms 
          
    probs:  ndarrray n x 1
          each entry is a probability for each of the n bins given by the surveyee, between 0 and 1
    
    returns
    -------
    moments: dict with 2 keys (more to be added in future)
            mean: empty or float, estimated mean 
            variance:  empty or float, estimated variance 
            std: empty or float, estimated standard deviation 
            irq1090: empty or float, estimated irq1090 
    
    """
    if math.fsum(probs)==1:
        print("probs sum up to 1")
        ## Check if all bins have the same probability (special case for which we need Uniform and not Beta distributions)
        all_equal = 0
        pos_bin = []
        # find non zero positions
        pos_entry = np.argwhere(probs!=0)
            
        for i in pos_entry:
            pos_bin.append(probs[i])
        pos_bin = np.hstack(pos_bin)   # clean    

        if len(pos_entry)>1:
            for i in range(len(pos_entry)-1) :
                if pos_entry[i+1] - pos_entry[i] !=1:
                    pos_bin = [] 
                
        if len(pos_bin)!=0:
            if np.all(pos_bin == pos_bin[0]):
                all_equal = 1
                
                
        ## Beta distributions 
        if sum([probs[i]>0 for i in range(len(bin)-1)])>=3 and all_equal == 0:
            print("at least three bins with positive probs")
            print("Beta Distribution")
            para_est=GeneralizedBetaEst(bin,probs)
            if len(para_est)==4:
                print('4 parameters')
                return GeneralizedBetaStats(para_est[0],para_est[1],para_est[2],para_est[3])
            if len(para_est)==2:
                print('2 parameters')
                return GeneralizedBetaStats(para_est[0],para_est[1],0,1)
        ## Triangle distributions
        if sum([probs[i]>0 for i in range(len(bin)-1)])==2 and all_equal == 0:
            print("Triangle Distribution")
            #print("There are two bins with positive probs")
            pprobadj = [i for i in range(1,len(bin)-3) if probs[i]>0 and probs[i+1]>0]   # from 1 to -3 bcz excluding the open-ended on the left/right
            if sum(pprobadj)>0:
                #print('The two intervals are adjacent and not open-ended')
                para_est=TriangleEst(bin,probs)
                return TriangleStats(para_est['lb'],para_est['ub'], para_est['mode'])
        ## Uniform distributions
        if sum([probs[i]>0 for i in range(len(bin)-1)])==1 and all_equal == 0:
            print('Only one interval with positive probs')
            print("Uniform distribution (1)")
            para_est= UniformEst(bin,probs)
            print(para_est)
            return UniformStats(para_est['lb'],para_est['ub'])
        if all_equal == 1:
            print("all bins have the same prob")
            print("Uniform distribution (2)")
            para_est= UniformEst(bin,probs)
            print(para_est)
            return UniformStats(para_est['lb'],para_est['ub'])            
        else:
            return {"mean":None,
                    'std': None,
                    'iqr1090':None,
                     "variance":None,
                    "variance":None,
                    "skewness":None,
                    "kurtosis":None}
    else:
        print("something went wrong")
        return {"mean":np.nan,
                'std': np.nan,
                'iqr1090':np.nan,
                "variance":np.nan,
                "skewness":np.nan,
                "kurtosis":np.nan}

### Estimation

In [10]:
### load data
BOP_data = pd.read_stata('C:/Users/adria/OneDrive/Dokumente/Corona and the Stockmarket/output/infexprob.dta')
#BOP_data = pd.read_csv('C:/Users/adria/OneDrive/Dokumente/Corona and the Stockmarket/output/infexprob.csv')
pd.DataFrame(BOP_data).head()

# Data treatment:
BOP_data = BOP_data.values 

Infl = BOP_data[:,1:]
Infl = Infl.astype(float)
print(Infl)

# divide by 100
for i in range(len(Infl)):
    Infl[i] = Infl[i]/100
    np.round(Infl[i],decimals=3)
np.round(Infl,decimals=3)
print(Infl)
index_aux = BOP_data[:,0]
index_aux = index_aux.astype(float)
index_aux = index_aux.reshape(len(index_aux),1)

print(index_aux)

# Make data frame
BOP_data = np.hstack((index_aux, Infl))
columns = ['index', 'infexprob_a', 'infexprob_b', 'infexprob_c', 'infexprob_d', 'infexprob_e', 'infexprob_f', 'infexprob_g', 'infexprob_h', 'infexprob_i', 'infexprob_j']
BOP_data = pd.DataFrame(BOP_data,columns=[columns])

[[50.  0.  0. ...  0.  0. 50.]
 [ 0.  0.  0. ... 10.  0.  0.]
 [ 0.  0.  0. ...  0.  0.  0.]
 ...
 [ 0.  0.  0. ...  0.  0.  0.]
 [ 0.  0.  0. ...  5.  0.  0.]
 [ 0.  0.  0. ...  0.  0.  0.]]
[[0.5  0.   0.   ... 0.   0.   0.5 ]
 [0.   0.   0.   ... 0.1  0.   0.  ]
 [0.   0.   0.   ... 0.   0.   0.  ]
 ...
 [0.   0.   0.   ... 0.   0.   0.  ]
 [0.   0.   0.   ... 0.05 0.   0.  ]
 [0.   0.   0.   ... 0.   0.   0.  ]]
[[700001.]
 [700002.]
 [700004.]
 ...
 [801243.]
 [801244.]
 [801248.]]


In [11]:
## survey-specific parameters 
#need 11 bins
# -14, -12, -8, -4, -2, 0, 2, 4, 8, 12, 14
BOP_bins=np.array([-14,-12,-8,-4,-2,0,2,4,8,12,14]) 
print("There are "+str(len(BOP_bins)-1)+" bins in BOP")

There are 10 bins in BOP


In [12]:
## creating positions 
index  = BOP_data.index
columns=['id','InflMean0','InflVar0', 'InflSD0', 'Infl90100']
IndBOP_moment_est = pd.DataFrame(index=index,columns=columns)

In [13]:
for i in range(len(index)):
    Infl0 = Infl[i,:]
    print(Infl0)
    IndBOP_moment_est['id'][i] = index_aux[i]
    print(IndBOP_moment_est['id'][i])
    if not np.isnan(Infl0).any():
        stats_est=SynDensityStat(BOP_bins,Infl0)
        IndBOP_moment_est['InflMean0'][i]=stats_est['mean']
        print(stats_est['mean'])
        IndBOP_moment_est['InflVar0'][i]=stats_est['variance']
        print(stats_est['variance']) 
        IndBOP_moment_est['InflSD0'][i]=stats_est['std']
        print(stats_est['std'])
        IndBOP_moment_est['Infl90100'][i]=stats_est['iqr1090']
        print(stats_est['iqr1090'])             

[0.5 0.  0.  0.  0.  0.  0.  0.  0.  0.5]
[700001.]
probs sum up to 1
Triangle Distribution
None
None
None
None
[0.  0.  0.  0.  0.  0.1 0.8 0.1 0.  0. ]
[700002.]
probs sum up to 1
at least three bins with positive probs
Beta Distribution
lower bound is 0
upper bound is 8
4 parameters
2.9807094778189347
0.5935211194758224
0.770403218760035
-0.35279974865509045
[0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
[700004.]
probs sum up to 1
all bins have the same prob
Uniform distribution (2)
length pos bins is 1
{'lb': 0, 'ub': 2}
1.0
0.3333333333333333
0.5773502691896257
1.6000000000000005
[0.   0.   0.   0.   0.   0.   0.   0.33 0.34 0.33]
[700007.]
probs sum up to 1
at least three bins with positive probs
Beta Distribution
lower bound is 4
upper bound is 14
4 parameters
7.44351078782725
3.6205238842025
1.9027674277752655
-0.8024778034880706
[0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1]
[700008.]
probs sum up to 1
all bins have the same prob
Uniform distribution (2)
length pos bins is > 1
{'lb': -14, 'ub': 

1.2754555058788397
1.1293606624452792
-0.3454954785630192
[0.05 0.   0.   0.   0.   0.   0.8  0.15 0.   0.  ]
[700064.]
probs sum up to 1
at least three bins with positive probs
Beta Distribution
lower bound is -14
upper bound is 8
4 parameters
14.226983647316057
1.3986701764805283
1.1826538701076188
-0.12297181569849436
[0.  0.  0.  0.  0.5 0.5 0.  0.  0.  0. ]
[700067.]
probs sum up to 1
all bins have the same prob
Uniform distribution (2)
length pos bins is > 1
{'lb': -2, 'ub': 2}
0.0
1.3333333333333333
1.1547005383792515
3.200000000000001
[0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
[700069.]
probs sum up to 1
all bins have the same prob
Uniform distribution (2)
length pos bins is 1
{'lb': -2, 'ub': 0}
-1.0
0.3333333333333333
0.5773502691896257
1.6000000000000005
[0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
[700071.]
probs sum up to 1
all bins have the same prob
Uniform distribution (2)
length pos bins is 1
{'lb': 0, 'ub': 2}
1.0
0.3333333333333333
0.5773502691896257
1.6000000000000005
[0.  0.  0.  0.  0.  

4 parameters
6.701827268458656
13.939063559394576
3.7335055322571273
-0.8010962388279488
[0.  0.  0.  0.  0.2 0.  0.8 0.  0.  0. ]
[700123.]
probs sum up to 1
Triangle Distribution
None
None
None
None
[0.   0.   0.   0.25 0.25 0.25 0.25 0.   0.   0.  ]
[700126.]
probs sum up to 1
all bins have the same prob
Uniform distribution (2)
length pos bins is > 1
{'lb': -4, 'ub': 4}
0.0
5.333333333333333
2.309401076758503
6.400000000000002
[0.01 0.02 0.03 0.04 0.2  0.3  0.3  0.08 0.01 0.01]
[700128.]
probs sum up to 1
at least three bins with positive probs
Beta Distribution
lower bound is -14
upper bound is 14
4 parameters
0.6666666666666666
0.05555555555555555
0.23570226039551584
-0.7232221713992888
[0.   0.5  0.25 0.25 0.   0.   0.   0.   0.   0.  ]
[700129.]
probs sum up to 1
at least three bins with positive probs
Beta Distribution
lower bound is -12
upper bound is -2
4 parameters
-2.1728208070495665
61.498342629234934
7.842087899866651
-0.8076724772163737
[0.24 0.12 0.08 0.04 0.02 0.02 0.

1.3144364707151737
1.1464887573435572
-0.5028572348051359
[0.   0.   0.   0.   0.25 0.5  0.25 0.   0.   0.  ]
[700189.]
probs sum up to 1
at least three bins with positive probs
Beta Distribution
lower bound is -2
upper bound is 4
4 parameters
1.9999709246980588
3.003110023081776
1.7329483613431116
-0.6994114063457937
[0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
[700191.]
probs sum up to 1
all bins have the same prob
Uniform distribution (2)
length pos bins is 1
{'lb': 0, 'ub': 2}
1.0
0.3333333333333333
0.5773502691896257
1.6000000000000005
[0.  0.  0.  0.  0.  0.9 0.1 0.  0.  0. ]
[700193.]
probs sum up to 1
Triangle Distribution
There are two bins with positive probs
The two intervals are adjacent and not open-ended
1.4624752955742644
0.35647233169417203
0.5970530392638262
1.7607606859185565
[0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
[700194.]
probs sum up to 1
all bins have the same prob
Uniform distribution (2)
length pos bins is 1
{'lb': 2, 'ub': 4}
3.0
0.3333333333333333
0.5773502691896257
1.60000000000

1.036684563884415
1.018177078844547
-0.458108936278705
[0.  0.  0.  0.  0.  0.8 0.2 0.  0.  0. ]
[700266.]
probs sum up to 1
Triangle Distribution
There are two bins with positive probs
The two intervals are adjacent and not open-ended
1.8090169943749475
0.5454237476562283
0.738528095373648
2.303357188869578
[0.15 0.05 0.1  0.05 0.15 0.1  0.05 0.1  0.15 0.1 ]
[700268.]
probs sum up to 1
at least three bins with positive probs
Beta Distribution
lower bound is -14
upper bound is 14
4 parameters
0.6666666666666666
0.05555555555555555
0.23570226039551584
-0.7232221713992888
[0.  0.  0.  0.  0.  0.8 0.2 0.  0.  0. ]
[700270.]
probs sum up to 1
Triangle Distribution
There are two bins with positive probs
The two intervals are adjacent and not open-ended
1.8090169943749475
0.5454237476562283
0.738528095373648
2.4931331211042296
[0.   0.   0.   0.   0.05 0.2  0.7  0.05 0.   0.  ]
[700271.]
probs sum up to 1
at least three bins with positive probs
Beta Distribution
lower bound is -2
upper bound

...

4 parameters
7.8916152110043924
2.022513863593863
1.4221511395044701
-0.22056990147972677
[0.  0.  0.  0.  0.  0.  0.  0.1 0.3 0.6]
[801039.]
probs sum up to 1
at least three bins with positive probs
Beta Distribution
lower bound is 4
upper bound is 14
4 parameters
8.674763994105398
2.037400254118228
1.4273753024759215
-0.7258336805019785
[0.07 0.07 0.08 0.15 0.13 0.14 0.13 0.1  0.08 0.05]
[801040.]
probs sum up to 1
at least three bins with positive probs
Beta Distribution
lower bound is -14
upper bound is 14
4 parameters
0.6666666666666666
0.05555555555555555
0.23570226039551584
-0.7232221713992888
[0.  0.  0.  0.  0.3 0.6 0.1 0.  0.  0. ]
[801041.]
probs sum up to 1
at least three bins with positive probs
Beta Distribution
lower bound is -2
upper bound is 4
4 parameters
1.4806776584964862
1.8682308282272337
1.3668324067811801
-0.5888360764333946
[0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
[801042.]
probs sum up to 1
all bins have the same prob
Uniform distribution (2)
length pos bins is 1
{'lb'

4 parameters
5.999956378463104
31.470817972326262
5.609885736120323
-0.5469041436614719
[0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
[801087.]
probs sum up to 1
all bins have the same prob
Uniform distribution (2)
length pos bins is 1
{'lb': 0, 'ub': 2}
1.0
0.3333333333333333
0.5773502691896257
1.6000000000000005
[0.   0.   0.   0.   0.   0.6  0.3  0.05 0.05 0.  ]
[801088.]
probs sum up to 1
at least three bins with positive probs
Beta Distribution
lower bound is 0
upper bound is 12
4 parameters
1.9435894013999997
2.1888633977458696
1.47948078654164
-0.4308545781394672
[0.  0.  0.  0.  0.4 0.6 0.  0.  0.  0. ]
[801089.]
probs sum up to 1
Triangle Distribution
There are two bins with positive probs
The two intervals are adjacent and not open-ended
-0.7207592200561266
1.2337551222534036
1.110745300351707
3.438865116214295
[0.  0.  0.  0.  0.5 0.5 0.  0.  0.  0. ]
[801092.]
probs sum up to 1
all bins have the same prob
Uniform distribution (2)
length pos bins is > 1
{'lb': -2, 'ub': 2}
0.0
1.333333333

1.4347347866695503
-0.4324510023072037
[0.  0.  0.  0.2 0.2 0.5 0.1 0.  0.  0. ]
[801140.]
probs sum up to 1
at least three bins with positive probs
Beta Distribution
lower bound is -4
upper bound is 4
4 parameters
2.1625257177612482
6.733018179913778
2.594806000438911
-0.6985699059746573
[0.  0.  0.2 0.2 0.2 0.3 0.1 0.  0.  0. ]
[801141.]
probs sum up to 1
at least three bins with positive probs
Beta Distribution
lower bound is -8
upper bound is 4
4 parameters
3.2407561016096444
20.917704785700874
4.5735877367446305
-0.7204216198268542
[0.  0.  0.  0.  0.  0.9 0.1 0.  0.  0. ]
[801142.]
probs sum up to 1
Triangle Distribution
There are two bins with positive probs
The two intervals are adjacent and not open-ended
1.4624752955742644
0.35647233169417203
0.5970530392638262
1.7388458524520793
[0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
[801143.]
probs sum up to 1
all bins have the same prob
Uniform distribution (2)
length pos bins is 1
{'lb': 2, 'ub': 4}
3.0
0.3333333333333333
0.5773502691896257
1.60

all bins have the same prob
Uniform distribution (2)
length pos bins is 1
{'lb': 4, 'ub': 8}
6.0
1.3333333333333333
1.1547005383792515
3.200000000000001
[0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
[801199.]
probs sum up to 1
all bins have the same prob
Uniform distribution (2)
length pos bins is 1
{'lb': 0, 'ub': 2}
1.0
0.3333333333333333
0.5773502691896257
1.6000000000000005
[0.  0.  0.  0.  0.5 0.5 0.  0.  0.  0. ]
[801200.]
probs sum up to 1
all bins have the same prob
Uniform distribution (2)
length pos bins is > 1
{'lb': -2, 'ub': 2}
0.0
1.3333333333333333
1.1547005383792515
3.200000000000001
[0.  0.  0.  0.  0.1 0.1 0.7 0.1 0.  0. ]
[801201.]
probs sum up to 1
at least three bins with positive probs
Beta Distribution
lower bound is -2
upper bound is 8
4 parameters
3.75508272160722
1.248075875572316
1.1171731627515566
-0.34233306678963515
[0.   0.01 0.02 0.07 0.4  0.4  0.07 0.02 0.01 0.  ]
[801203.]
probs sum up to 1
at least three bins with positive probs
Beta Distribution
lower bound is -12

length pos bins is 1
{'lb': -4, 'ub': -2}
-3.0
0.3333333333333333
0.5773502691896257
1.6000000000000005
[0.   0.   0.   0.   0.   0.8  0.15 0.05 0.   0.  ]
[801244.]
probs sum up to 1
at least three bins with positive probs
Beta Distribution
lower bound is 0
upper bound is 8
4 parameters
1.1074145378828215
1.757506538304228
1.3257098243221357
-0.5668004256776581
[0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
[801248.]
probs sum up to 1
all bins have the same prob
Uniform distribution (2)
length pos bins is 1
{'lb': 0, 'ub': 2}
1.0
0.3333333333333333
0.5773502691896257
1.6000000000000005


In [14]:
# show and export
print(IndBOP_moment_est)
IndBOP_moment_est.to_excel('Python_output.xlsx')

              id           InflMean0            InflVar0   InflSD0 Infl90100
0     [700001.0]                None                None      None      None
1     [700002.0]  2.9807094778189347  0.5935211194758224  0.770403   -0.3528
2     [700004.0]                 1.0            0.333333   0.57735       1.6
3     [700007.0]    7.44351078782725     3.6205238842025  1.902767 -0.802478
4     [700008.0]                 0.0           65.333333  8.082904      22.4
...          ...                 ...                 ...       ...       ...
1712  [801241.0]            1.462475            0.356472  0.597053  1.780682
1713  [801242.0]    8.68421483533903  42.107339826127465  6.489017 -0.711473
1714  [801243.0]                -3.0            0.333333   0.57735       1.6
1715  [801244.0]  1.1074145378828215   1.757506538304228   1.32571   -0.5668
1716  [801248.0]                 1.0            0.333333   0.57735       1.6

[1717 rows x 5 columns]


### Check missing cases

In [15]:
### load data
BOP_data_missing = pd.read_stata('C:/Users/adria/OneDrive/Dokumente/Corona and the Stockmarket/output/infexprob_missing.dta')
pd.DataFrame(BOP_data_missing).head()

# Data treatment:
BOP_data_missing = BOP_data_missing.values 

Infl_missing = BOP_data_missing[:,1:]
Infl_missing = Infl_missing.astype(float)
print(Infl_missing)

# divide by 100
for i in range(len(Infl_missing)):
    Infl_missing[i] = Infl_missing[i]/100
np.round(Infl_missing,decimals=3)

print(Infl_missing)
index_aux_missing = BOP_data_missing[:,0]
index_aux_missing = index_aux_missing.astype(float)
index_aux_missing = index_aux_missing.reshape(len(index_aux_missing),1)

print(index_aux_missing)

# Make data frame
BOP_data_missing = np.hstack((index_aux_missing, Infl_missing))
columns = ['index', 'infexprob_a', 'infexprob_b', 'infexprob_c', 'infexprob_d', 'infexprob_e', 'infexprob_f', 'infexprob_g', 'infexprob_h', 'infexprob_i', 'infexprob_j']
BOP_data_missing = pd.DataFrame(BOP_data_missing,columns=[columns])

## creating positions 
index_missing  = BOP_data_missing.index
columns_missing=['id','InflMean0','InflVar0', 'InflSD0', 'Infl90100']
IndBOP_moment_est_missing = pd.DataFrame(index=index_missing,columns=columns_missing)

[[ 0.  0. 30.  0.  0.  0. 20. 50.  0.  0.]
 [50.  0.  0.  0.  0.  0.  0.  0.  0. 50.]
 [ 0.  0.  0.  0.  0.  0. 60. 20. 20.  0.]
 [ 0.  0.  0. 50.  0. 50.  0.  0.  0.  0.]
 [ 0.  0.  0. 20.  0.  0.  0. 80.  0.  0.]
 [ 0.  0.  0.  0. 20.  0. 80.  0.  0.  0.]
 [ 0.  0.  0. 10. 50.  0.  0.  0. 40.  0.]
 [ 0.  0.  0.  0. 60.  0.  0.  0.  0. 40.]
 [ 0.  0.  0.  0. 20.  0. 80.  0.  0.  0.]
 [ 0.  0.  0.  0.  0. 70. 20.  0. 10.  0.]
 [ 0.  0.  0.  0.  0.  0. 10. 10. 10. 70.]
 [ 0. 40.  0.  0.  0.  0.  0.  0.  0. 60.]
 [ 0.  0.  0. 50.  0. 50.  0.  0.  0.  0.]
 [ 0.  0.  0. 50.  0.  0. 50.  0.  0.  0.]
 [20.  0.  0.  0.  0.  0.  0.  0.  0. 80.]
 [20.  0.  0.  0.  0.  0.  0.  0. 80.  0.]
 [ 0.  0.  0.  0.  0. 70. 10. 10.  5.  5.]
 [ 0.  0.  0.  0.  0.  0. 60. 20. 15.  5.]
 [ 0.  0.  0.  0. 20.  0. 80.  0.  0.  0.]
 [40.  0.  0.  0.  0.  0. 60.  0.  0.  0.]
 [ 0.  0.  0.  0. 60.  0. 40.  0.  0.  0.]
 [ 0.  0.  0. 50.  0.  0. 50.  0.  0.  0.]
 [ 0.  0. 50.  0.  0.  0. 50.  0.  0.  0.]
 [ 0.  0.  

In [16]:
for i in range(len(index_missing)):
    Infl0 = Infl_missing[i,:]
    print(Infl0)
    Infl0 = np.round(Infl0,decimals=3)
    print(type(Infl0))
    print(np.sum(Infl0))
    print(math.fsum(Infl0))
    IndBOP_moment_est['id'][i] = index_aux[i]
    print(IndBOP_moment_est['id'][i])
    if not np.isnan(Infl0).any():
        stats_est=SynDensityStat(BOP_bins,Infl0)
#         if stats_est.size>0: # no idea what that is
        IndBOP_moment_est['InflMean0'][i]=stats_est['mean']
        print(stats_est['mean'])
        IndBOP_moment_est['InflVar0'][i]=stats_est['variance']
        print(stats_est['variance']) 
        IndBOP_moment_est['InflSD0'][i]=stats_est['std']
        print(stats_est['std'])
        IndBOP_moment_est['Infl90100'][i]=stats_est['iqr1090']
        print(stats_est['iqr1090'])   

[0.  0.  0.3 0.  0.  0.  0.2 0.5 0.  0. ]
<class 'numpy.ndarray'>
1.0
1.0
[700001.]
probs sum up to 1
at least three bins with positive probs
Beta Distribution
lower bound is -8
upper bound is 8
None
None
None
None
[0.5 0.  0.  0.  0.  0.  0.  0.  0.  0.5]
<class 'numpy.ndarray'>
1.0
1.0
[700002.]
probs sum up to 1
Triangle Distribution
None
None
None
None
[0.  0.  0.  0.  0.  0.  0.6 0.2 0.2 0. ]
<class 'numpy.ndarray'>
1.0
1.0
[700004.]
probs sum up to 1
at least three bins with positive probs
Beta Distribution
lower bound is 2
upper bound is 12
None
None
None
None
[0.  0.  0.  0.5 0.  0.5 0.  0.  0.  0. ]
<class 'numpy.ndarray'>
1.0
1.0
[700007.]
probs sum up to 1
Triangle Distribution
None
None
None
None
[0.  0.  0.  0.2 0.  0.  0.  0.8 0.  0. ]
<class 'numpy.ndarray'>
1.0
1.0
[700008.]
probs sum up to 1
Triangle Distribution
None
None
None
None
[0.  0.  0.  0.  0.2 0.  0.8 0.  0.  0. ]
<class 'numpy.ndarray'>
1.0
1.0
[700010.]
probs sum up to 1
Triangle Distribution
None
None
None

Triangle Distribution
None
None
None
None
[0.  0.  0.  0.  0.1 0.  0.9 0.  0.  0. ]
<class 'numpy.ndarray'>
1.0
1.0
[700095.]
probs sum up to 1
Triangle Distribution
None
None
None
None
[0.  0.  0.  0.5 0.  0.  0.5 0.  0.  0. ]
<class 'numpy.ndarray'>
1.0
1.0
[700096.]
probs sum up to 1
Triangle Distribution
None
None
None
None
[0.  0.  0.  0.  0.5 0.  0.  0.  0.  0.5]
<class 'numpy.ndarray'>
1.0
1.0
[700098.]
probs sum up to 1
Triangle Distribution
None
None
None
None
[0.  0.  0.  0.5 0.  0.5 0.  0.  0.  0. ]
<class 'numpy.ndarray'>
1.0
1.0
[700099.]
probs sum up to 1
Triangle Distribution
None
None
None
None
[0.  0.  0.  0.  0.  0.  0.  0.  0.8 0.2]
<class 'numpy.ndarray'>
1.0
1.0
[700101.]
probs sum up to 1
Triangle Distribution
None
None
None
None
[0.  0.  0.  0.1 0.  0.  0.9 0.  0.  0. ]
<class 'numpy.ndarray'>
1.0
1.0
[700102.]
probs sum up to 1
Triangle Distribution
None
None
None
None
[0.  0.  0.  0.  0.5 0.  0.5 0.  0.  0. ]
<class 'numpy.ndarray'>
1.0
1.0
[700104.]
probs sum 

In [17]:
## Problem: uniform and triangular distribution does not produce 90-10
sim_bins= np.array([-14,-12,-8,-4,-2,0,2,4,8,12,14]) 
sim_probs = np.array([0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1])
para_est= UniformEst(sim_bins,sim_probs)
print(para_est)
UniformStats(para_est['lb'],para_est['ub']) 

# x = np.linspace(uniform.ppf(para_est['lb']),uniform.ppf(para_est['ub']), 100)
# p10 = np.percentile(x, 10)
# p90 = np.percentile(x, 90)
# iqr1090 = p90-p10
# print(x)

length pos bins is > 1
{'lb': -14, 'ub': 14}


{'mean': 0.0,
 'variance': 65.33333333333333,
 'std': 8.082903768654761,
 'iqr1090': 22.4,
 'skewness': 0,
 'kurtosis': -0.8333333333333334}

In [18]:
## Problem: uniform and triangular distribution does not produce 90-10
sim_bins= np.array([-14,-12,-8,-4,-2,0,2,4,8,12,14]) 
sim_probs = np.array([0,0,0,0.2,0.8,0,0,0,0,0])
para_est= TriangleEst(sim_bins,sim_probs)
print(para_est)
TriangleStats(para_est['lb'],para_est['ub'], para_est['mode'])


There are two bins with positive probs
The two intervals are adjacent and not open-ended
{'lb': -3.618033988749895, 'ub': 0, 'height': 0.552786404500042, 'mode': -3.6180339887498953}


{'mean': -1.8090169943749475,
 'variance': 0.5454237476562283,
 'std': 0.738528095373648,
 'iqr1090': 2.0354041163431726,
 'skewness': 0,
 'kurtosis': -0.6}

In [19]:
c = 1.1
x = np.linspace(triang.ppf(0.01, c),
                triang.ppf(0.99, c), 100)
quantiles = quantile(x,[0.1, 0.9])
print(quantiles)

[nan, nan]
